<a href="https://colab.research.google.com/github/milad9595naeimi/Economic_Specialization/blob/main/Economic_Specialization_Excluding_Iceland_NewZealand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## RCA and Normalized RCA Calculation

The Revealed Comparative Advantage (RCA) is a measure introduced by Balassa (1965) to identify a country's relative advantage in a specific export sector. RCA is calculated as follows:

$$
\text{RCA}_{\text{Country, Sector, Year}} = \frac{\left( \frac{\text{Exports of Country in Sector in Year}}{\text{Total Exports of Country in Year}} \right)}{\left( \frac{\text{Global Exports in Sector in Year}}{\text{Global Total Exports in Year}} \right)}
$$

Where:

- $ \text{Exports of Country in Sector in Year} $ represents the export value of a specific sector for a given country in that year.
- $ \text{Total Exports of Country in Year} $ is the total export value of the country across all sectors in the given year.
- $ \text{Global Exports in Sector in Year} $ is the total global exports in that specific sector for the given year.
- $ \text{Global Total Exports in Year} $ is the total global exports across all sectors in the given year.

If the RCA value is greater than 1, the country has a comparative advantage in that sector; if RCA is less than 1, it has a comparative disadvantage.

The **Normalized RCA** is calculated to scale the RCA between -1 and +1 using the following formula:

$$
\text{RCA Normalized} = \frac{\text{RCA} - 1}{\text{RCA} + 1}
$$

This normalization adjusts the RCA so that:

- Positive values (RCA > 0) indicate comparative advantage.
- Negative values (RCA < 0) indicate comparative disadvantage.





## Regression Model Equations

The general form of the regression model used to analyze the relationship between normalized RCA and various factors is as follows:

$$
\text{RCA}_{\text{Normalized},it} = \beta_0 + \beta_1 \cdot \text{Classification}_{it} + \beta_2 \cdot \text{Real_GDP}_{it} + \beta_3 \cdot \text{KOFGI}_{it} + \epsilon_{it}
$$

Where:

- **RCA_Normalized{it}\)**: The normalized RCA for country \(i\) in sector \(t\).
- **Classification\(_{it}\)**: A categorical variable representing the country's variety of capitalism.
- **Real\_GDP\_2015\_Billion\(_{it}\)**: The real GDP of country \(i\) in constant 2015 USD.
- **KOFGI\(_{it}\)**: The KOF Globalization Index for country \(i\) in year \(t\).
An extended model includes a dummy variable to account for the USA's unique characteristics:

$$
\text{RCA}_{\text{Normalized},it} = \beta_0 + \beta_1 \cdot \text{Classification}_{it} + \beta_2 \cdot \text{Real_GDP}_{it} + \beta_3 \cdot \text{KOFGI}_{it} + \beta_4 \cdot \text{US_Dummy}_{it} + \epsilon_{it}
$$

Where:

- **US\_Dummy_{it}\)**: A binary variable that is 1 if the country is the USA and 0 otherwise.



In [ ]:
import pandas as pd
import requests
from io import StringIO
import statsmodels.formula.api as smf

# Step 1: Load and prepare the dataset for 1991-1998
df_high_1991_1998 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/high_tech_1991_1998.csv')
df_high_1991_1998['Value'] = df_high_1991_1998['Value'] / 1_000_000  # Convert to billions

# Step 2: Load GDP data from API for 1991-1998
api_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.CAN+AUS+IRL+USA+GBR+SWE+FIN+DNK+NLD+CHE+NOR+BEL+AUT+DEU+JPN.B1GQ_R.USD_PPP.?startPeriod=1991&endPeriod=1998"
headers = {"Accept": "application/vnd.sdmx.data+csv; charset=utf-8; version=2"}
response = requests.get(api_url, headers=headers)
response.raise_for_status()

# Convert CSV response to DataFrame
data_io = StringIO(response.content.decode('utf-8'))
df_gdp_1991_1998 = pd.read_csv(data_io, sep=',')
df_gdp_1991_1998 = df_gdp_1991_1998.rename(columns={"REF_AREA": "COU", "TIME_PERIOD": "Time", "OBS_VALUE": "Real_GDP_2015_Billion"})
df_gdp_1991_1998 = df_gdp_1991_1998[['COU', 'Time', 'Real_GDP_2015_Billion']]

# Merge export data and GDP data
df_high_1991_1998['Time'] = df_high_1991_1998['Time'].astype(int)
df_gdp_1991_1998['Time'] = df_gdp_1991_1998['Time'].astype(int)
merged_df_1991_1998 = df_high_1991_1998.merge(df_gdp_1991_1998, on=['COU', 'Time'], how='left')

# Step 3: Load and merge KOFGI data for 1991-1998
kofgi_file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/KOFGI_2023_public.xlsx'
kofgi_data = pd.read_excel(kofgi_file_path)
filtered_kofgi = kofgi_data[(kofgi_data['code'].isin(merged_df_1991_1998['COU'])) & (kofgi_data['year'].between(1991, 1998))]
filtered_kofgi = filtered_kofgi.rename(columns={'code': 'COU', 'year': 'Time'})[['COU', 'Time', 'KOFGI']]

# Merge KOFGI data with the merged data
final_df_high_1991_1998 = merged_df_1991_1998.merge(filtered_kofgi, on=['COU', 'Time'], how='left')

# Step 4: Calculate RCA while preserving all existing columns for 1991-1998
# Summarize total exports by country, sector, and world without losing any columns
df_aggregated_1991_1998 = final_df_high_1991_1998.groupby(['sector_tech', 'COU', 'Time'], as_index=False).agg({
    'Value': 'sum',  # Summing up export values
    'Real_GDP_2015_Billion': 'mean',  # Averaging GDP if applicable
    'KOFGI': 'mean',  # Averaging KOFGI if applicable
    'Classification': 'first'  # Preserving Classification (taking the first occurrence)
})

# Summarize total exports by country, sector, and world for 1991-1998
country_total_exports_1991_1998 = final_df_high_1991_1998.groupby(['COU', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Country_Total_Exports'})
sector_total_exports_1991_1998 = final_df_high_1991_1998.groupby(['sector_tech', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Sector_Total_Exports'})
world_total_exports_1991_1998 = final_df_high_1991_1998.groupby('Time', as_index=False)['Value'].sum().rename(columns={'Value': 'World_Total_Exports'})

# Merge total exports by country, sector, and world back into the data
df_aggregated_1991_1998 = df_aggregated_1991_1998.merge(country_total_exports_1991_1998, on=['COU', 'Time'])
df_aggregated_1991_1998 = df_aggregated_1991_1998.merge(sector_total_exports_1991_1998, on=['sector_tech', 'Time'])
df_aggregated_1991_1998 = df_aggregated_1991_1998.merge(world_total_exports_1991_1998, on='Time')

# Step 5: Calculate RCA and RCA Normalized for 1991-1998
df_aggregated_1991_1998['RCA'] = (df_aggregated_1991_1998['Value'] / df_aggregated_1991_1998['Country_Total_Exports']) / \
                                 (df_aggregated_1991_1998['Sector_Total_Exports'] / df_aggregated_1991_1998['World_Total_Exports'])

df_aggregated_1991_1998['RCA_Normalized'] = (df_aggregated_1991_1998['RCA'] - 1) / (df_aggregated_1991_1998['RCA'] + 1)

# Step 6: Generate US dummy variable for 1991-1998
df_aggregated_1991_1998['US_Dummy'] = df_aggregated_1991_1998['COU'].apply(lambda x: 1 if x == 'USA' else 0)

# Step 7: Ensure 'Classification' is treated as a categorical variable for 1991-1998
df_aggregated_1991_1998['Classification'] = df_aggregated_1991_1998['Classification'].astype('category')

# Recode 'Classification' with 'LME' as the reference category
df_aggregated_1991_1998['Classification'] = df_aggregated_1991_1998['Classification'].cat.reorder_categories(['LME', 'Liberalized_CME', 'Stable_CME'])

# Step 8: Run Regression Models for 1991-1998
# Formula for Model 3: RCA_Normalized ~ Classification + Real GDP + KOFGI
formula_1_1991_1998 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI'
model_1_1991_1998 = smf.ols(formula_1_1991_1998, data=df_aggregated_1991_1998).fit()

# Print the summary of the regression results for Model 1
print("Model 1 Regression Results for 1991-1998:")
print(model_1_1991_1998.summary())

# Formula for Model 4: RCA_Normalized ~ Classification + Real GDP + KOFGI + US Dummy
formula_2_1991_1998 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI + US_Dummy'
model_2_1991_1998 = smf.ols(formula_2_1991_1998, data=df_aggregated_1991_1998).fit()

# Print the summary of the regression results for Model 2
print("Model 2 Regression Results for 1991-1998:")
print(model_2_1991_1998.summary())

# Step 9: Export the final data with RCA and RCA_Normalized for 1991-1998
csv_file_path_1991_1998 = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/high_tech_1991_1998_final_with_regressions.csv'
df_aggregated_1991_1998.to_csv(csv_file_path_1991_1998, index=False)

print(f"Final data with RCA and regression results for 1991-1998 has been exported to {csv_file_path_1991_1998}")


Model 1 Regression Results for 1991-1998:
                            OLS Regression Results                            
Dep. Variable:         RCA_Normalized   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.293
Date:                Sun, 25 Aug 2024   Prob (F-statistic):             0.0583
Time:                        23:39:40   Log-Likelihood:                -198.05
No. Observations:                 570   AIC:                             406.1
Df Residuals:                     565   BIC:                             427.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [ ]:
import pandas as pd
import requests
from io import StringIO
import statsmodels.formula.api as smf

# Step 1: Load and prepare the dataset
df_high_2000_2007 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/high_tech_2000_2007.csv')
df_high_2000_2007['Value'] = df_high_2000_2007['Value'] / 1_000_000  # Convert to millions

# Step 2: Load GDP data from API
api_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.CAN+AUS+IRL+USA+GBR+SWE+FIN+DNK+NLD+CHE+NOR+BEL+AUT+DEU+JPN.B1GQ_R.USD_PPP.?startPeriod=2000&endPeriod=2007"
headers = {"Accept": "application/vnd.sdmx.data+csv; charset=utf-8; version=2"}
response = requests.get(api_url, headers=headers)
response.raise_for_status()

# Convert CSV response to DataFrame
data_io = StringIO(response.content.decode('utf-8'))
df_gdp_2000_2007 = pd.read_csv(data_io, sep=',')
df_gdp_2000_2007 = df_gdp_2000_2007.rename(columns={"REF_AREA": "COU", "TIME_PERIOD": "Time", "OBS_VALUE": "Real_GDP_2015_Billion"})
df_gdp_2000_2007 = df_gdp_2000_2007[['COU', 'Time', 'Real_GDP_2015_Billion']]

# Merge export data and GDP data
df_high_2000_2007['Time'] = df_high_2000_2007['Time'].astype(int)
df_gdp_2000_2007['Time'] = df_gdp_2000_2007['Time'].astype(int)
merged_df_2000_2007 = df_high_2000_2007.merge(df_gdp_2000_2007, on=['COU', 'Time'], how='left')

# Step 3: Load and merge KOFGI data
kofgi_file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/KOFGI_2023_public.xlsx'
kofgi_data = pd.read_excel(kofgi_file_path)
filtered_kofgi = kofgi_data[(kofgi_data['code'].isin(merged_df_2000_2007['COU'])) & (kofgi_data['year'].between(2000, 2007))]
filtered_kofgi = filtered_kofgi.rename(columns={'code': 'COU', 'year': 'Time'})[['COU', 'Time', 'KOFGI']]

# Merge KOFGI data with the merged data
final_df_high_2000_2007 = merged_df_2000_2007.merge(filtered_kofgi, on=['COU', 'Time'], how='left')

# Step 4: Calculate RCA while preserving all existing columns
# Summarize total exports by country, sector, and world without losing any columns
df_aggregated = final_df_high_2000_2007.groupby(['sector_tech', 'COU', 'Time'], as_index=False).agg({
    'Value': 'sum',  # Summing up export values
    'Real_GDP_2015_Billion': 'mean',  # Averaging GDP if applicable
    'KOFGI': 'mean',  # Averaging KOFGI if applicable
    # Add additional columns you want to preserve with an appropriate aggregation method
    'Classification': 'first'  # Preserving Classification (taking the first occurrence)
})

# Summarize total exports by country, sector, and world
country_total_exports = final_df_high_2000_2007.groupby(['COU', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Country_Total_Exports'})
sector_total_exports = final_df_high_2000_2007.groupby(['sector_tech', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Sector_Total_Exports'})
world_total_exports = final_df_high_2000_2007.groupby('Time', as_index=False)['Value'].sum().rename(columns={'Value': 'World_Total_Exports'})

# Merge total exports by country, sector, and world back into the data
df_aggregated = df_aggregated.merge(country_total_exports, on=['COU', 'Time'])
df_aggregated = df_aggregated.merge(sector_total_exports, on=['sector_tech', 'Time'])
df_aggregated = df_aggregated.merge(world_total_exports, on='Time')

# Step 5: Calculate RCA and RCA Normalized
df_aggregated['RCA'] = (df_aggregated['Value'] / df_aggregated['Country_Total_Exports']) / \
                       (df_aggregated['Sector_Total_Exports'] / df_aggregated['World_Total_Exports'])

df_aggregated['RCA_Normalized'] = (df_aggregated['RCA'] - 1) / (df_aggregated['RCA'] + 1)

# Step 6: Generate US dummy variable
df_aggregated['US_Dummy'] = df_aggregated['COU'].apply(lambda x: 1 if x == 'USA' else 0)

# Step 7: Ensure 'Classification' is treated as a categorical variable
df_aggregated['Classification'] = df_aggregated['Classification'].astype('category')

# Recode 'Classification' with 'LME' as the reference category
df_aggregated['Classification'] = df_aggregated['Classification'].cat.reorder_categories(['LME', 'Liberalized_CME', 'Stable_CME'])

# Step 8: Run Regression Models
# Formula for Model 3: RCA_Normalized ~ Classification + Real GDP + KOFGI
formula_3 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI'
model_3_2000_2007 = smf.ols(formula_3, data=df_aggregated).fit()

# Print the summary of the regression results for Model 3
print("Model 3 Regression Results:")
print(model_3_2000_2007.summary())

# Formula for Model 4: RCA_Normalized ~ Classification + Real GDP + KOFGI + US Dummy
formula_4 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI + US_Dummy'
model_4_2000_2007 = smf.ols(formula_4, data=df_aggregated).fit()

# Print the summary of the regression results for Model 4
print("Model 4 Regression Results:")
print(model_4_2000_2007.summary())

# Step 9: Export the final data with RCA and RCA_Normalized
csv_file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/high_tech_2000_2007_final_with_regressions.csv'
df_aggregated.to_csv(csv_file_path, index=False)

print(f"Final data with RCA and regression results has been exported to {csv_file_path}")


Model 3 Regression Results:
                            OLS Regression Results                            
Dep. Variable:         RCA_Normalized   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     4.604
Date:                Sun, 25 Aug 2024   Prob (F-statistic):            0.00114
Time:                        23:45:11   Log-Likelihood:                -219.06
No. Observations:                 600   AIC:                             448.1
Df Residuals:                     595   BIC:                             470.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------

In [ ]:
import pandas as pd
import requests
from io import StringIO
import statsmodels.formula.api as smf

# Step 1: Load and prepare the dataset for 1991-1998
df_medium_1991_1998 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/medium_high_tech_1991_1998.csv')
df_medium_1991_1998['Value'] = df_medium_1991_1998['Value'] / 1_000_000  # Convert to millions

# Step 2: Load GDP data from API for 1991-1998
api_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.CAN+AUS+IRL+USA+GBR+SWE+FIN+DNK+NLD+CHE+NOR+BEL+AUT+DEU+JPN.B1GQ_R.USD_PPP.?startPeriod=1991&endPeriod=1998"
headers = {"Accept": "application/vnd.sdmx.data+csv; charset=utf-8; version=2"}
response = requests.get(api_url, headers=headers)
response.raise_for_status()

# Convert CSV response to DataFrame
data_io = StringIO(response.content.decode('utf-8'))
df_gdp_1991_1998 = pd.read_csv(data_io, sep=',')
df_gdp_1991_1998 = df_gdp_1991_1998.rename(columns={"REF_AREA": "COU", "TIME_PERIOD": "Time", "OBS_VALUE": "Real_GDP_2015_Billion"})
df_gdp_1991_1998 = df_gdp_1991_1998[['COU', 'Time', 'Real_GDP_2015_Billion']]

# Merge export data and GDP data
df_medium_1991_1998['Time'] = df_medium_1991_1998['Time'].astype(int)
df_gdp_1991_1998['Time'] = df_gdp_1991_1998['Time'].astype(int)
merged_df_medium_1991_1998 = df_medium_1991_1998.merge(df_gdp_1991_1998, on=['COU', 'Time'], how='left')

# Step 3: Load and merge KOFGI data for 1991-1998
kofgi_file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/KOFGI_2023_public.xlsx'
kofgi_data = pd.read_excel(kofgi_file_path)
filtered_kofgi = kofgi_data[(kofgi_data['code'].isin(merged_df_medium_1991_1998['COU'])) & (kofgi_data['year'].between(1991, 1998))]
filtered_kofgi = filtered_kofgi.rename(columns={'code': 'COU', 'year': 'Time'})[['COU', 'Time', 'KOFGI']]

# Merge KOFGI data with the merged data
final_df_medium_1991_1998 = merged_df_medium_1991_1998.merge(filtered_kofgi, on=['COU', 'Time'], how='left')

# Step 4: Calculate RCA while preserving all existing columns for 1991-1998
# Summarize total exports by country, sector, and world without losing any columns
df_aggregated_medium_1991_1998 = final_df_medium_1991_1998.groupby(['sector_tech', 'COU', 'Time'], as_index=False).agg({
    'Value': 'sum',  # Summing up export values
    'Real_GDP_2015_Billion': 'mean',  # Averaging GDP if applicable
    'KOFGI': 'mean',  # Averaging KOFGI if applicable
    'Classification': 'first'  # Preserving Classification (taking the first occurrence)
})

# Summarize total exports by country, sector, and world for 1991-1998
country_total_exports_medium_1991_1998 = final_df_medium_1991_1998.groupby(['COU', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Country_Total_Exports'})
sector_total_exports_medium_1991_1998 = final_df_medium_1991_1998.groupby(['sector_tech', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Sector_Total_Exports'})
world_total_exports_medium_1991_1998 = final_df_medium_1991_1998.groupby('Time', as_index=False)['Value'].sum().rename(columns={'Value': 'World_Total_Exports'})

# Merge total exports by country, sector, and world back into the data
df_aggregated_medium_1991_1998 = df_aggregated_medium_1991_1998.merge(country_total_exports_medium_1991_1998, on=['COU', 'Time'])
df_aggregated_medium_1991_1998 = df_aggregated_medium_1991_1998.merge(sector_total_exports_medium_1991_1998, on=['sector_tech', 'Time'])
df_aggregated_medium_1991_1998 = df_aggregated_medium_1991_1998.merge(world_total_exports_medium_1991_1998, on='Time')

# Step 5: Calculate RCA and RCA Normalized for 1991-1998
df_aggregated_medium_1991_1998['RCA'] = (df_aggregated_medium_1991_1998['Value'] / df_aggregated_medium_1991_1998['Country_Total_Exports']) / \
                                        (df_aggregated_medium_1991_1998['Sector_Total_Exports'] / df_aggregated_medium_1991_1998['World_Total_Exports'])

df_aggregated_medium_1991_1998['RCA_Normalized'] = (df_aggregated_medium_1991_1998['RCA'] - 1) / (df_aggregated_medium_1991_1998['RCA'] + 1)

# Step 6: Generate US dummy variable for 1991-1998
df_aggregated_medium_1991_1998['US_Dummy'] = df_aggregated_medium_1991_1998['COU'].apply(lambda x: 1 if x == 'USA' else 0)

# Step 7: Ensure 'Classification' is treated as a categorical variable for 1991-1998
df_aggregated_medium_1991_1998['Classification'] = df_aggregated_medium_1991_1998['Classification'].astype('category')

# Recode 'Classification' with 'LME' as the reference category
df_aggregated_medium_1991_1998['Classification'] = df_aggregated_medium_1991_1998['Classification'].cat.reorder_categories(['LME', 'Liberalized_CME', 'Stable_CME'])

# Step 8: Run Regression Models for 1991-1998
# Formula for Model 5: RCA_Normalized ~ Classification + Real GDP + KOFGI
formula_5_1991_1998 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI'
model_5_1991_1998 = smf.ols(formula_5_1991_1998, data=df_aggregated_medium_1991_1998).fit()

# Print the summary of the regression results for Model 5
print("Model 5 Regression Results for 1991-1998:")
print(model_5_1991_1998.summary())

# Formula for Model 6: RCA_Normalized ~ Classification + Real GDP + KOFGI + US Dummy
formula_6_1991_1998 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI + US_Dummy'
model_6_1991_1998 = smf.ols(formula_6_1991_1998, data=df_aggregated_medium_1991_1998).fit()

# Print the summary of the regression results for Model 6
print("Model 6 Regression Results for 1991-1998:")
print(model_6_1991_1998.summary())

# Step 9: Export the final data with RCA and RCA_Normalized for 1991-1998
csv_file_path_medium_1991_1998 = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/medium_tech_1991_1998_final_with_regressions.csv'
df_aggregated_medium_1991_1998.to_csv(csv_file_path_medium_1991_1998, index=False)

print(f"Final data with RCA and regression results for 1991-1998 has been exported to {csv_file_path_medium_1991_1998}")


Model 5 Regression Results for 1991-1998:
                            OLS Regression Results                            
Dep. Variable:         RCA_Normalized   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     4.066
Date:                Sun, 25 Aug 2024   Prob (F-statistic):            0.00294
Time:                        23:40:27   Log-Likelihood:                -71.226
No. Observations:                 570   AIC:                             152.5
Df Residuals:                     565   BIC:                             174.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

Import and Conduct Regression Analysis for Medium High Tech 2000-2007

In [ ]:
import pandas as pd
import requests
from io import StringIO
import statsmodels.formula.api as smf

# Step 1: Load and prepare the dataset for 2000-2007
df_medium_2000_2007 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/medium_high_tech_2000_2007.csv')
df_medium_2000_2007['Value'] = df_medium_2000_2007['Value'] / 1_000_000  # Convert to millions

# Step 2: Load GDP data from API for 2000-2007
api_url = "https://sdmx.oecd.org/public/rest/data/OECD.SDD.NAD,DSD_NAAG@DF_NAAG_I,1.0/A.CAN+AUS+IRL+USA+GBR+SWE+FIN+DNK+NLD+CHE+NOR+BEL+AUT+DEU+JPN.B1GQ_R.USD_PPP.?startPeriod=2000&endPeriod=2007"
headers = {"Accept": "application/vnd.sdmx.data+csv; charset=utf-8; version=2"}
response = requests.get(api_url, headers=headers)
response.raise_for_status()

# Convert CSV response to DataFrame
data_io = StringIO(response.content.decode('utf-8'))
df_gdp_2000_2007 = pd.read_csv(data_io, sep=',')
df_gdp_2000_2007 = df_gdp_2000_2007.rename(columns={"REF_AREA": "COU", "TIME_PERIOD": "Time", "OBS_VALUE": "Real_GDP_2015_Billion"})
df_gdp_2000_2007 = df_gdp_2000_2007[['COU', 'Time', 'Real_GDP_2015_Billion']]

# Merge export data and GDP data
df_medium_2000_2007['Time'] = df_medium_2000_2007['Time'].astype(int)
df_gdp_2000_2007['Time'] = df_gdp_2000_2007['Time'].astype(int)
merged_df_medium_2000_2007 = df_medium_2000_2007.merge(df_gdp_2000_2007, on=['COU', 'Time'], how='left')

# Step 3: Load and merge KOFGI data for 2000-2007
kofgi_file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/KOFGI_2023_public.xlsx'
kofgi_data = pd.read_excel(kofgi_file_path)
filtered_kofgi = kofgi_data[(kofgi_data['code'].isin(merged_df_medium_2000_2007['COU'])) & (kofgi_data['year'].between(2000, 2007))]
filtered_kofgi = filtered_kofgi.rename(columns={'code': 'COU', 'year': 'Time'})[['COU', 'Time', 'KOFGI']]

# Merge KOFGI data with the merged data
final_df_medium_2000_2007 = merged_df_medium_2000_2007.merge(filtered_kofgi, on=['COU', 'Time'], how='left')

# Step 4: Calculate RCA while preserving all existing columns for 2000-2007
# Summarize total exports by country, sector, and world without losing any columns
df_aggregated_medium_2000_2007 = final_df_medium_2000_2007.groupby(['sector_tech', 'COU', 'Time'], as_index=False).agg({
    'Value': 'sum',  # Summing up export values
    'Real_GDP_2015_Billion': 'mean',  # Averaging GDP if applicable
    'KOFGI': 'mean',  # Averaging KOFGI if applicable
    'Classification': 'first'  # Preserving Classification (taking the first occurrence)
})

# Summarize total exports by country, sector, and world for 2000-2007
country_total_exports_medium_2000_2007 = final_df_medium_2000_2007.groupby(['COU', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Country_Total_Exports'})
sector_total_exports_medium_2000_2007 = final_df_medium_2000_2007.groupby(['sector_tech', 'Time'], as_index=False)['Value'].sum().rename(columns={'Value': 'Sector_Total_Exports'})
world_total_exports_medium_2000_2007 = final_df_medium_2000_2007.groupby('Time', as_index=False)['Value'].sum().rename(columns={'Value': 'World_Total_Exports'})

# Merge total exports by country, sector, and world back into the data
df_aggregated_medium_2000_2007 = df_aggregated_medium_2000_2007.merge(country_total_exports_medium_2000_2007, on=['COU', 'Time'])
df_aggregated_medium_2000_2007 = df_aggregated_medium_2000_2007.merge(sector_total_exports_medium_2000_2007, on=['sector_tech', 'Time'])
df_aggregated_medium_2000_2007 = df_aggregated_medium_2000_2007.merge(world_total_exports_medium_2000_2007, on='Time')

# Step 5: Calculate RCA and RCA Normalized for 2000-2007
df_aggregated_medium_2000_2007['RCA'] = (df_aggregated_medium_2000_2007['Value'] / df_aggregated_medium_2000_2007['Country_Total_Exports']) / \
                                        (df_aggregated_medium_2000_2007['Sector_Total_Exports'] / df_aggregated_medium_2000_2007['World_Total_Exports'])

df_aggregated_medium_2000_2007['RCA_Normalized'] = (df_aggregated_medium_2000_2007['RCA'] - 1) / (df_aggregated_medium_2000_2007['RCA'] + 1)

# Step 6: Generate US dummy variable for 2000-2007
df_aggregated_medium_2000_2007['US_Dummy'] = df_aggregated_medium_2000_2007['COU'].apply(lambda x: 1 if x == 'USA' else 0)

# Step 7: Ensure 'Classification' is treated as a categorical variable for 2000-2007
df_aggregated_medium_2000_2007['Classification'] = df_aggregated_medium_2000_2007['Classification'].astype('category')

# Recode 'Classification' with 'LME' as the reference category
df_aggregated_medium_2000_2007['Classification'] = df_aggregated_medium_2000_2007['Classification'].cat.reorder_categories(['LME', 'Liberalized_CME', 'Stable_CME'])

# Step 8: Run Regression Models for 2000-2007
# Formula for Model 7: RCA_Normalized ~ Classification + Real GDP + KOFGI
formula_7_medium_2000_2007 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI'
model_7_medium_2000_2007 = smf.ols(formula_7_medium_2000_2007, data=df_aggregated_medium_2000_2007).fit()

# Print the summary of the regression results for Model 7
print("Model 7 Regression Results for 2000-2007:")
print(model_7_medium_2000_2007.summary())

# Formula for Model 8: RCA_Normalized ~ Classification + Real GDP + KOFGI + US Dummy
formula_8_medium_2000_2007 = 'RCA_Normalized ~ C(Classification) + Real_GDP_2015_Billion + KOFGI + US_Dummy'
model_8_medium_2000_2007 = smf.ols(formula_8_medium_2000_2007, data=df_aggregated_medium_2000_2007).fit()

# Print the summary of the regression results for Model 8
print("Model 8 Regression Results for 2000-2007:")
print(model_8_medium_2000_2007.summary())

# Step 9: Export the final data with RCA and RCA_Normalized for 2000-2007
csv_file_path_medium_2000_2007 = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/medium_tech_2000_2007_final_with_regressions.csv'
df_aggregated_medium_2000_2007.to_csv(csv_file_path_medium_2000_2007, index=False)

print(f"Final data with RCA and regression results for 2000-2007 has been exported to {csv_file_path_medium_2000_2007}")


Model 7 Regression Results for 2000-2007:
                            OLS Regression Results                            
Dep. Variable:         RCA_Normalized   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     9.307
Date:                Sun, 25 Aug 2024   Prob (F-statistic):           2.67e-07
Time:                        23:40:46   Log-Likelihood:                -82.203
No. Observations:                 600   AIC:                             174.4
Df Residuals:                     595   BIC:                             196.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------

In [ ]:
#!pip install pandas statsmodels tabulate
import pandas as pd
from statsmodels.iolib.summary2 import summary_col

# Assuming you have 8 regression models stored in model_1, model_2, ..., model_8
models = [model_1_1991_1998, model_2_1991_1998, model_3_2000_2007, model_4_2000_2007, model_5_1991_1998, model_6_1991_1998, model_7_medium_2000_2007, model_8_medium_2000_2007]
model_names = ['Model 1', 'Model 2', 'Model 3', 'Model 4', 'Model 5', 'Model 6', 'Model 7', 'Model 8']

# Create a summary table
summary = summary_col(models, stars=True, float_format='%0.4f', model_names=model_names, info_dict={'R-squared': lambda x: f"{x.rsquared:.4f}"})

# Convert summary to a DataFrame for better formatting control
df_summary = summary.tables[0]

# Clean up the table format for professional presentation
df_summary.reset_index(inplace=True)
df_summary.columns = ['Variables'] + model_names
df_summary.set_index('Variables', inplace=True)

# Add significance stars manually if needed
def add_stars(value, pvalue):
    if pvalue < 0.01:
        return f"{value}***"
    elif pvalue < 0.05:
        return f"{value}**"
    elif pvalue < 0.1:
        return f"{value}*"
    else:
        return f"{value}"

# Apply the function to the DataFrame
for model, name in zip(models, model_names):
    pvalues = model.pvalues
    for var in pvalues.index:
        value = df_summary.at[var, name]
        df_summary.at[var, name] = add_stars(value, pvalues[var])

# Save the table to an HTML file
html_content = df_summary.to_html(classes='table table-striped', border=0)

file_path = '/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/panel_regression_models_results.html'
with open(file_path, 'w') as f:
    f.write(html_content)

file_path



'/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/panel_regression_models_results.html'

In [ ]:
import pandas as pd

# Step 1: Load and prepare the dataset for 1991-1998
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Exports Data Economic Specialization/Excluding_Iceland_NewZealand/high_tech_1991_1998.csv')
df['Value'] = df['Value'] / 1_000_000  # Convert to billions

# Step 1: Extract USA's export value for Aircraft_and_spacecraft sector in 1991
usa_aircraft_exports_1991 = df[(df['COU'] == 'USA') & (df['Time'] == 1991) & (df['sector_tech'] == 'Aircraft_and_spacecraft')]['Value'].sum()

# Step 2: Calculate USA's total exports across all sectors in 1991
usa_total_exports_1991 = df[(df['COU'] == 'USA') & (df['Time'] == 1991)]['Value'].sum()

# Step 3: Calculate global exports for the Aircraft_and_spacecraft sector in 1991
global_aircraft_exports_1991 = df[(df['Time'] == 1991) & (df['sector_tech'] == 'Aircraft_and_spacecraft')]['Value'].sum()

# Step 4: Calculate global total exports across all sectors in 1991
global_total_exports_1991 = df[df['Time'] == 1991]['Value'].sum()

# Step 5: Calculate RCA for USA in 1991 for Aircraft_and_spacecraft
rca_usa_aircraft_1991 = (usa_aircraft_exports_1991 / usa_total_exports_1991) / (global_aircraft_exports_1991 / global_total_exports_1991)

# Print the calculated RCA value
print(f"RCA for USA in 1991 for Aircraft_and_spacecraft sector: {rca_usa_aircraft_1991}")

# Calculate Normalized RCA
rca_normalized = (rca_usa_aircraft_1991 - 1) / (rca_usa_aircraft_1991 + 1)

# Print the calculated normalized RCA value
print(f"Normalized RCA for USA in 1991 for Aircraft_and_spacecraft sector: {rca_normalized}")


RCA for USA in 1991 for Aircraft_and_spacecraft sector: 1.7463283026563863
Normalized RCA for USA in 1991 for Aircraft_and_spacecraft sector: 0.2717549471177573
